# Classifying Cats Versus Dogs with a Convolutional Neural Network

We're going to use online image repositories and the `tensorflow` package to determine whether a given image is of a cat versus a dog.

This is roughly modified from the first part of the tutorial here: https://www.tensorflow.org/tutorials/images/classification

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import pathlib

%matplotlib inline

## Let's grab some data and fit our model

In [ ]:
### let's grab the data -- this is the quick data set
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=dataset_url, extract=True)
valid_dir = pathlib.Path(data_dir.replace('.zip','')+'/validation').with_suffix('')
data_dir = pathlib.Path(data_dir.replace('.zip','')+'/train').with_suffix('')


image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
data_dir

In [ ]:
# this one is 25000 images -- works better!
# download link from:
# https://www.dropbox.com/scl/fi/vr0acehjcsz0iyq2yo8gr/PetImages.zip?rlkey=brbmz8mn7g0r6l8obv3tykguh&dl=0
# and unpack in your local directory, then uncomment the following line

#data_dir = pathlib.Path('PetImages').with_suffix('')

#image_count = len(list(data_dir.glob('*/*.jpg')))
#print(image_count)

In [ ]:
### let's see what images we're actually looking at
cats = list(data_dir.glob('cats/*'))
cats_1 = PIL.Image.open(str(cats[0]))
cats_2 = PIL.Image.open(str(cats[1]))
    
dogs = list(data_dir.glob('dogs/*'))
dogs_1 = PIL.Image.open(str(dogs[0]))

plt.imshow(dogs_1)

cats_1.size

In [ ]:
plt.imshow(cats_1)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

# 80% will be a "training" sample
# 20% will be a "validation" sample, which we'll test to use if it works
# training:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# validation:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# reminder of our classifications
class_names = train_ds.class_names
print(class_names)

In [ ]:
 # make sure we keep images in memory
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)    

# scale from 0-1 instead of 0-255 (normal RGB colors)
normalization_layer = layers.Rescaling(1./255)

### now the hard part
num_classes = len(class_names)

In [ ]:
### convolution layer: isolates features, many fewer parameters than working with pixels individually
### pooling layer: reducing spatial size of convolved feature, maxes computationally easier
###    here, we're returning max value of portion of the image returned by the kernel
###    max value selects the "most important" feature of the pooled region
### activation: 0 if negative, equal to value if positive 
###     (ignores negative-valued neurons, allows training to select useful vs non-useful features)
### dense layer: fully-connected, all the features together for training
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

epochs=10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

In [ ]:
### now test it out on a validation image
dog_path = str(valid_dir)+'/dogs/dog.2000.jpg'
cat_path = str(valid_dir)+'/cats/cat.2000.jpg'
test_image = PIL.Image.open(dog_path)
test_image_cat = PIL.Image.open(cat_path)
test_image_cat

In [ ]:
img = tf.keras.utils.load_img(
    cat_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

## Now, can it tell me whether my photos are of cats???

In [ ]:
import glob
easy_cats = glob.glob('EasyCats/*.jpg')
for e in easy_cats:
    img = tf.keras.utils.load_img(
        e, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
hard_cats = glob.glob('DifficultCats/*.jpg')
for h in hard_cats:
    img = tf.keras.utils.load_img(
        h, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

## Well shit...let's try some data augmentation to handle rotation, translation

And a few more iterations

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(12):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, name="outputs")
    ])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

## Let's See if we Did Any Better...

In [ ]:
import glob
easy_cats = glob.glob('EasyCats/*.jpg')
for e in easy_cats:
    img = tf.keras.utils.load_img(
        e, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
hard_cats = glob.glob('DifficultCats/*.jpg')
for h in hard_cats:
    img = tf.keras.utils.load_img(
        h, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()